In [33]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [34]:
import pandas as pd
import json

# функция, которая позволяет считать тренировочный набор данных и вернуть в виде Датафрейм
def get_X_y(data_json_file):
    X, y = [], []
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        #print(json_list[0])
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            #print(item['passage'].keys())
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    X.append(text+' Query: '+query+' Answer: '+a['text'])
                    y.append(a['label'])
    return X, y

X_train, y_train = get_X_y("/content/gdrive/MyDrive/Анализ Данных/Kaggle Text Understanding/train.jsonl")

train_df = pd.DataFrame({
    'text': X_train,
    'label': y_train
})

# в колонке 'text'
# лежит сам текст, вопрос, ответ на этот вопрос
# так как к одному тексту может быть несколько ответов, то похожих строк может быть несколько
train_df

,text,label
0,"(1) Но люди не могут существовать без природы,...",1
1,"(1) Но люди не могут существовать без природы,...",0
2,"(1) Но люди не могут существовать без природы,...",0
3,"(1) Но люди не могут существовать без природы,...",1
4,"(1) Но люди не могут существовать без природы,...",0
...,...,...
11945,(1) Известный американский боец смешанных един...,0
11946,(1) Известный американский боец смешанных един...,1
11947,(1) Известный американский боец смешанных един...,0
11948,(1) Известный американский боец смешанных един...,0


In [35]:
import pandas as pd

# функция которая позволяет считать данные из теста и записать их в Датафрейм
def get_X_test(data_json_file):
    X1 = []
    X2 = []
    X3 = []
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    X1.append(text) #текст
                    X2.append(query) #вопрос
                    X3.append(a['text']) #ответ на вопрос
    return X1, X2, X3

X1, X2, X3 = get_X_test('/content/gdrive/MyDrive/Анализ Данных/Kaggle Text Understanding/val_wtihout.jsonl')

test_df = pd.DataFrame({
    'text': X1,
    'query': X2,
    'answer': X3
})

test_df
# В колонке text лежит: текст, вопрос и ответ на этот вопрос
# Вам нужно реализовать алгоритм, который вернет, ответ на вопрос верный или нет

,text,query,answer
0,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,Так перевозили особо важных заключенных.
1,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что был эмигрантом."
2,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что он сам вырыл себе землянку."
3,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что он побывал на пересылке Красная Пр..."
4,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что он был особо важным заключённым и ..."
...,...,...,...
2230,"""(1) Канадский порноактер Лука Рокко Маньотта,...",За что осуждён Рокко Маньотта?,Изнасилование.
2231,"""(1) Канадский порноактер Лука Рокко Маньотта,...",За что осуждён Рокко Маньотта?,Рокко похитил любовника и присвоил его деньги.
2232,"""(1) Канадский порноактер Лука Рокко Маньотта,...",За что осуждён Рокко Маньотта?,Убийство и расчленение тела.
2233,"""(1) Канадский порноактер Лука Рокко Маньотта,...",За что осуждён Рокко Маньотта?,Рокко убил любовника и расчленил его тело.


In [36]:
train_df['text'] = train_df['text'].str.lower()
test_df['text'] = test_df['text'].str.lower()
test_df['query'] = test_df['query'].str.lower()
test_df['answer'] = test_df['answer'].str.lower()

In [37]:
sings = list("?.,;:!-()12'34567890/{}[]()'<>?&*$@-_")

for sing in sings:
  train_df['text'] = train_df['text'].str.replace(sing, '')
  test_df['text'] =  test_df['text'].str.replace(sing, '')
  test_df['query'] = test_df['query'].str.replace(sing, '')
  test_df['answer'] = test_df['answer'].str.replace(sing, '')

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()

train = train_df['text'].tolist()
labels = train_df['label'].tolist()
test_ = test_df["text"].tolist()

X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.33, random_state=42)

#X_train_vector = count_vectorizer.fit_transform(X_train).toarray()
#X_test_vector = count_vectorizer.transform(X_test).toarray()

X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)
test_counts=count_vectorizer.transform(test_)

In [39]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_train_counts, y_train)
y_predicted_counts = clf.predict(X_test_counts)

In [40]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, roc_auc_score

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')

    roc_auc = roc_auc_score(y_test, y_predicted_counts)
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)

    return accuracy, precision, recall, f1, roc_auc

accuracy, precision, recall, f1, roc_auc = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f, roc_auc = %.3f" % (accuracy, precision, recall, f1, roc_auc))

accuracy = 0.498, precision = 0.461, recall = 0.498, f1 = 0.442, roc_auc = 0.473


In [41]:
y_predicted_test = clf.predict(test_counts)
y_test_pred = pd.DataFrame(y_predicted_test, columns=['label'])
y_test_pred.reset_index(inplace=True)

y_test_pred

,index,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
2230,2230,0
2231,2231,0
2232,2232,0
2233,2233,0


In [42]:
from google.colab import files
y_test_pred.to_csv('yana.csv', index=False)
files.download("yana.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>